# Google Map 店家評論爬蟲
### 作者：黃鈺淇

## 引入套件以及需要的函式

In [1]:
import os
import re
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from bs4 import BeautifulSoup as Soup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

In [2]:
search = "大安區 星巴克"

## 開啟 Google Map 並查詢

In [3]:
# 設定工作目錄
os.chdir("/Users/andiology/Downloads/Google Map Review 爬蟲") # 要改
# 設定要前往的網址
url = 'https://www.google.com.tw/maps?hl=zh-TW&tab=rl&authuser=0'  
# 透過 Browser Driver 開啟 Chrome
browser = webdriver.Chrome('./chromedriver')  
# 前往該網址
browser.get(url) 
# 填入搜尋內容
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'tactile-searchbox-input')))
# 網頁元素定位 
search_input = browser.find_elements_by_class_name('tactile-searchbox-input')[0]
# 輸入搜尋內容 
search_input.send_keys(search)
# 搜尋
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'searchbox-searchbutton-container')))
# 網頁元素定位 
search_click = browser.find_elements_by_class_name('searchbox-searchbutton-container')[0]
# 點擊搜尋鍵 
search_click.click()
# 成功搜尋
print("Finish Searching!")

Finish Searching!


In [4]:
i = 1 # 第幾個店家

### While 迴圈
當存在下一頁就 繼續執行

In [5]:
# Next = True
# while Next:

In [6]:
# 取得網頁元素框架（搜尋結果）
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'section-result')))
soup = Soup(browser.page_source,"lxml")
store_frame = soup.find_all(class_="section-result")

In [7]:
# 找所有店家名稱
store_name = []
for stores in store_frame:
    new_store = stores.get('aria-label')
    if (new_store != None) & (new_store not in store_name):
        store_name.append(new_store)
print("店家名稱：")
for sn in store_name:
    print(sn)

店家名稱：
STARBUCKS 星巴克 (興和門市)
STARBUCKS 星巴克 (敦和門市)
STARBUCKS 星巴克 (長興門市)
Starbucks
STARBUCKS 星巴克 (辛亥門市)
STARBUCKS 星巴克 (敦富門市)
STARBUCKS 星巴克 (通化門市)
STARBUCKS 星巴克 (仁愛延吉門市)
STARBUCKS 星巴克 (雲和門市)
STARBUCKS RESERVE™ 星巴克龍門新概念店
STARBUCKS 星巴克 (復安門市)
STARBUCKS 星巴克 (崇興門市)
STARBUCKS 星巴克 (崇光門市)
STARBUCKS 星巴克 (華視門市)
STARBUCKS 星巴克 (公館門市)
STARBUCKS 星巴克 (同領門市)
STARBUCKS 星巴克 (忠孝延吉門市)


## 進入個別店家

### For 迴圈
執行所有店家

In [8]:
# for n in range(len(store_name)):
n = 0 # 第幾個店家（每20頁會重置數字）

### 點擊進入個別店家 

In [9]:
# 抓個別店家資料
WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'section-result')))
store_click = browser.find_elements_by_class_name('section-result')[n]
store_click.click()

## 進入所有評論

![](https://imgur.com/krhWTav.png)

In [31]:
# 點擊查看所有評論
# 原本的 class name 中存在空格：'allxGeDnJMl__button allxGeDnJMl__button-text'，會讓程式抓不到
# 所以空格的部分要改成「.」才能成功抓取
review_click = browser.find_elements_by_class_name('allxGeDnJMl__button.allxGeDnJMl__button-text')
actions = ActionChains(browser)
for rc in range(len(review_click)):
    actions.move_to_element(review_click[rc]).perform()
    if "更多評論" in review_click[rc].text:
        review_click[rc].click()

![](https://imgur.com/nB8HnZN.png)

In [32]:
# 評論選單
menu_click = browser.find_elements_by_class_name('iRxY3GoUYUY__button.gm2-hairline-border.section-action-chip-button')
for mc in menu_click:
    if mc.text == "排序":
        print(mc.text)
        mc.click()

排序


In [33]:
# 選擇評論類型
data_index = 1 # 最新
category_click = browser.find_elements_by_class_name('action-menu-entry')
print(category_click[data_index].text)
category_click[data_index].click()

最新


![](https://i.imgur.com/SGdiWqh.png)

### 問題一：
### 在評論類型中選擇有提到「相關字詞」的評論
### 如："座位"、"舒適" 等

In [34]:
# 有提到的內容的評論（ex.提到 "座位"、"舒適"）
mentioned_index = 1
mentioned_click = browser.find_elements_by_class_name('tuPVDR7ouq5__button.gm2-hairline-border.section-input-chip-button')
print(mentioned_click[mentioned_index].text)
mentioned_click[mentioned_index].click()

座位
21


![](https://imgur.com/a1UtNna.png)

In [35]:
# 評論分頁下滑
scroll_time = 3 # 看你要滑幾次
for st in range(scroll_time):
    pane = browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')
    browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
    print(st+1,'scroll'*(st+1))
    time.sleep(2)

1 scroll
2 scrollscroll
3 scrollscrollscroll


![](https://imgur.com/FYVR2KR.png)

In [36]:
# 先將所有照片都點開來
# 因為一則評論中的照片若超過四張，就會被縮減起來，所以把它點擊開來才能加載並抓取
all_photo = browser.find_elements_by_class_name('section-review-photo')
for ap in all_photo:
    ap.click()

## 評論資訊

### 預設產出資料list及時間

In [37]:
all_title_review = []
all_subtitle_review = []
all_star_review = []
all_date_review = []
all_text_review = []
all_photo_review = []
today = datetime.today().strftime("%Y-%m-%d")

![](https://imgur.com/q2KRx8M.png)

In [38]:
# 找出所有評論的元素
soup = Soup(browser.page_source,"lxml")
all_reviews = soup.find_all(class_ = 'section-review ripple-container GLOBAL__gm2-body-2')

In [39]:
# 找出所有評論資訊加入list中
# for ar in all_reviews:
ar = all_reviews[0] # 第幾則評論

![](https://imgur.com/9ZQwEEe.png)

In [40]:
# 評論者名稱
all_title_review.append(ar.find(class_ = "section-review-title").text)

![](https://imgur.com/D1rTBVS.png)

In [20]:
# 評論者代稱＆評論數
try:
    subtitle_review = ar.find(class_ = "section-review-subtitle").text
except:
    subtitle_review = []
if ' ・' in subtitle_review: 
    all_subtitle_review.append(subtitle_review)
elif ' 在地嚮導' in subtitle_review:
    all_subtitle_review.append(subtitle_review.strip(' 在地嚮導'))
else:
    all_subtitle_review.append(subtitle_review)

![](https://imgur.com/irh0dIf.png)

In [21]:
# 評論星數
all_star_review.append(str(ar.find(class_ = "section-review-stars").get('aria-label').strip().strip("顆星")))

![](https://imgur.com/cZPUzTE.png)

![](https://imgur.com/YyQTUFx.png)

### 問題二：
### 爬取評論時間以及評論內容

In [22]:
# 評論時間
all_date_review.append(ar.find(class_ = "section-review-publish-date").text)

In [23]:
# 評論內容
all_text_review.append(ar.find(class_ = "section-review-text").text)

![](https://imgur.com/p4r1zac.png)

In [24]:
# 評論照片
photos = ar.find_all(class_ = "section-review-photo")
photo_urls = []
for ph in photos:
    photo_urls.append(ph.get('style').strip('background-image:url(').strip(')'))
all_photo_review.append(photo_urls)

### 問題三：
### 將所有評論資料合併成一個 Data Frame
### 根據以下欄位：
    'Store'
    'Today'
    'Reviewer Title'
    'Reviewer Subtitle'
    'Star Reviews'
    'Date'
    'Text'
    'Photos'

In [25]:
# 將所有評論資料存成DataFrame
reviews = pd.DataFrame({
    'Store':store_name[n],
    'Today':today,
    'Reviewer Title':all_title_review,
    'Reviewer Subtitle':all_subtitle_review,
    'Star Reviews':all_star_review,
    'Date':all_date_review,
    'Text':all_text_review,
    'Photos':all_photo_review
    })

In [26]:
reviews

,Store,Today,Reviewer Title,Reviewer Subtitle,Star Reviews,Date,Text,Photos
0,STARBUCKS 星巴克 (興和門市),2020-12-22,Chen Ada,在地嚮導 ・82 則評論,2,1 個月前,之前在別家分店都會叫「飲料名＋姓（先生/小姐）」，這次我以為會叫。\n\n所以點了一杯飲料外...,[]


### 問題四：
### 輸出 csv files
### 所有評論建檔以 "0\_店家名稱\_所有評論.csv" 為格式
### 請先創建好「個別店家評論」資料夾

In [27]:
# 輸出 csv files
reviews.to_csv("個別店家評論/"+ str(i) +"_"+ store_name[n] +"_所有評論.csv", encoding = 'UTF-8-sig')
i += 1

![](https://imgur.com/nSy1CzF.png)

## 返回個別店家頁面

In [28]:
# 點擊返回店家頁面
print('return to store')
WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'ozj7Vb3wnYq__action-button-clickable')))
BacktoStore_btn = browser.find_elements_by_class_name('ozj7Vb3wnYq__action-button-clickable')[0]
BacktoStore_btn.click()

return to store


## 返回搜尋頁面

In [29]:
# 點擊返回搜尋結果
print('return to search result')
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'section-back-to-list-button.blue-link.noprint')))
back_btn = browser.find_elements_by_class_name('section-back-to-list-button.blue-link.noprint')[0]
back_btn.click()

return to search result


### For 迴圈結尾

In [30]:
# 點擊下一頁按鈕
try:
    WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.ID, 'n7lv7yjyC35__section-pagination-button-next')))
    next_page = browser.find_elements_by_class_name('n7lv7yjyC35__button.noprint')[1]
    next_page.click()
    # Next = True
    print('Next page!')
    time.sleep(2)
except:
    # Next = False
    print('No next page!')

Next page!


### While 迴圈結尾